In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("Lab_6_matrices_kinematics.ipynb")

# Using matrices to position parts of a robot arm

The goal of this part of the assignment is to use matrices to position a robot arm in space. In the lab we'll just
position each component independently and rotate them by angles. In the homework we'll pose all of the links.


Slides: https://docs.google.com/presentation/d/1Ut5RnIKU8DF8k_joGXp4tJ1FzBKNIX8JYRE9wkIP_qE/edit?

Moving from the lecture activity to the lab:
- Each component of the arm is stored as a dictionary. These are put into a list (see create_arm_geometry in arm_routines.py)
- The points for the objects are stored in the object's dicitonary - check out create_arm_component() in arm_routines.py and the two routines used to make the point matrices (points_in_a_square and points_in_a_wedge in arm_routines.py)   
- The object dictionary stores the points, the transformation matrix that changes the geometry/points to be the desired shape ("Matrix_shape") and a second matrix that puts the compoment in the right place in space ("Matrix_pose) and some additional stuff (name, color, lengths, angles)
- Theere are two plot routines. The first plots the object with just the Matrix_shape matrix. The second uses both to place the object in space (Matrix_pose @ Matrix_shape)

Note: These **plot routines are called at the end of each question in this jupyter notebook**; they should at least run even if you haven't edited the code (they'll just draw objects with the identity matrix). You should jump to the end to the plots to visualize what you've done as you work through the problems.

In [ ]:
# The usual imports
import numpy as np
import matplotlib.pyplot as plt

# These are the routines used in the lecture activity - we'll re-use them here
import matrix_routines as mt

In [ ]:
# These are the routines you'll edit for the lab. You can edit them in the.py file OR copy them in from the .py file to here
#  If you do the latter, you'll probably want to copy them back to arm_routines.py for the homework
# If you decide to copy the functions in here, make sure to copy ALL of the functions in arm_routines.py to here, 
#   then comment out this line and do a restart
from arm_routines import points_in_a_square, points_in_a_wedge, \
    matrix_shape_base, matrix_shape_link, matrix_shape_palm, matrix_shape_finger

In [ ]:
# These are the routines in arm_routines.py you'll need, but you don't need to change/edit them
from arm_routines import create_arm_component, create_arm_geometry, plot_arm_components

In [ ]:
# These commands will force JN to actually re-load the external file when you re-execute the import command
%load_ext autoreload
%autoreload 2

## Step 1: Shape the base and the links

Use matrices to take two basic shapes (a square and a wedge) and re-shape them into the geometry for the arm, gripper.

Yes, you could just create these basic shapes with the "correct" XYs, but we'll use a matrix to transform the
basic shape (square, wedge) to the correct size and shape. This way, we only have to make a square and a wedge and then just use matrices to shape them.

This is actually what most packages (eg, solidworks) do when you make a model. Each part of the model is defined in a "canonical" location, then transformed to the desired position/scale/rotation using a matarix. This is
 *before* calculating the matrix that positions the part based on the, eg, joint angles

For all of these, you should be creating a matrix that consists of a scale followed by a rotate (maybe) followed by a translate

See slides for what the resulting re-positioned shapes look like; a reminder that there is a plot function at the bottom of this JN that you can use at any time.

TODO step 1: Copy your code to make the square points into **points_in_a_square** in **arm_routines.py** (the wedge is done for you).

TODO step 2: Edit **matrix_shape_base, matrix_shape_link, matrix_shape_palm, and (optional) matrix_shape_finger**

These functions do the equivalent of what we did with **mat_transform_square** in the lecture activity.

The following tests/cells breaks out each of those routines and checks the results. The function **create_arm_geometry** puts all of those functions together to create the geometry for the entire arm.

In [ ]:
# The sizes for all of the components
base_size_param = (0.5, 1.0)
link_sizes_param = [(0.5, 0.25), (0.3, 0.1), (0.2, 0.05)]
palm_width_param = 0.1
finger_size_param = (0.075, 0.025)

In [ ]:
# Check the returned values
np.set_printoptions(precision=4, floatmode='fixed')  # Print out with 4 digits of precision

# This is the base of the robot
base_obj = create_arm_component(name="Base", pts=points_in_a_wedge(), color="darkturquoise")

# Here is where the matrix that moves the base (which is a wedge) to the right size/place is set
matrix_shape_base(base_obj=base_obj, base_height=base_size_param[0], base_width=base_size_param[1])

print(f"Matrix\n{base_obj['Matrix_shape']}")
print(f"Pts transformed\n{base_obj['Matrix_shape'] @ base_obj['Pts']}")

mat_base_check = np.array([[0.5, 0.0, 0], [0.0, 0.25, 0.25], [0.0, 0.0, 1.0]])
assert(np.all(np.isclose(base_obj["Matrix_shape"], mat_base_check)))

In [ ]:
link_obj = create_arm_component(name=f"Link", pts=points_in_a_square(), color="black")

# Set the matrix that scales/translates the square to the right place
matrix_shape_link(link_obj=link_obj, link_length=link_sizes_param[0][0], link_width=link_sizes_param[0][1])

print(f"Matrix\n{link_obj['Matrix_shape']}")
print(f"Pts transformed\n{link_obj['Matrix_shape'] @ link_obj['Pts']}")

mat_link1_check = np.array([[0.25, 0.0, 0.25], [0.0, 0.125, 0.0], [0.0, 0.0, 1.0]])
assert(np.all(np.isclose(link_obj["Matrix_shape"], mat_link1_check)))

In [ ]:
palm_obj = create_arm_component(name="Palm", pts=points_in_a_square(), color="tomato")
# Sets the Matrix_scale key
matrix_shape_palm(palm_obj, palm_width=palm_width_param)

print(f"Matrix\n{palm_obj['Matrix_shape']}")
print(f"Pts transformed\n{palm_obj['Matrix_shape'] @ palm_obj['Pts']}")

mat_palm_check = np.array([[0.005, 0.0, 0.0], [0.0, 0.05, 0.0], [0.0, 0.0, 1.0]])
assert(np.all(np.isclose(palm_obj["Matrix_shape"], mat_palm_check)))

In [ ]:
# Top finger - we'll do the bottom one in the next cell
finger_top_obj = create_arm_component(name="Finger_top", pts=points_in_a_wedge(), color="black")

matrix_shape_finger(finger_top_obj, palm_width=palm_width_param, finger_size=finger_size_param, b_is_top=True)

print(f"Matrix\n{finger_top_obj['Matrix_shape']}")
print(f"Pts transformed\n{finger_top_obj['Matrix_shape'] @ finger_top_obj['Pts']}")

mat_finger_top_check = np.array([[0.0375, 0.0, 0.0375], [0.0, 0.0125, 0.05], [0.0, 0.0, 1.0]])
assert(np.all(np.isclose(finger_top_obj["Matrix_shape"], mat_finger_top_check)))

In [ ]:
finger_bot_obj = create_arm_component(name="Finger_top", pts=points_in_a_wedge(), color="black")

matrix_shape_finger(finger_bot_obj, palm_width=0.1, finger_size=finger_size_param, b_is_top=False)

print(f"Matrix\n{finger_bot_obj['Matrix_shape']}")
print(f"Pts transformed\n{finger_bot_obj['Matrix_shape'] @ finger_bot_obj['Pts']}")

mat_finger_bot_check = np.array([[0.0375, 0.0, 0.0375], [0.0, 0.0125, -0.05], [0.0, 0.0, 1.0]])
assert(np.all(np.isclose(finger_bot_obj["Matrix_shape"], mat_finger_bot_check)))


In [ ]:
grader.check("Matrix_position")

# Put it all together to make the entire arm plus gripper

This should just "work" if you did the previous part correctly (no TODOs). 

In [ ]:

# This function calls each of the set_transform_xxx functions, and puts the results
# in a list (the gripper - the last element - is a list)
arm_geometry = create_arm_geometry(base_size_param, link_sizes_param, palm_width_param, finger_size_param)
if len(arm_geometry) != 5:
    print("Wrong number of components, should be 5, got {len(arm_geometry)}")
if len(arm_geometry[-1]) != 3:
    print("Wrong number of gripper components, should be 3, got {len(arm_geometry[-1])}")

# Print the base and the links
for link in arm_geometry[:-1]:
    print(f"Link {link['Name']}")
    print(f"  Matrix (shape)\n   {link['Matrix_shape']}")

# ... and the gripper
print("Gripper:")
for link in arm_geometry[-1]:
    print(f" Part: {link['Name']}")
    print(f"  Matrix (shape)\n   {link['Matrix_shape']}")


In [ ]:
# Should show all 5 components, the base, 3 links, and the gripper
# Step 1 - note, comment out this one if you don't want both drawn on top of each other when you do step 2
fig, axs = plt.subplots(1, len(arm_geometry), figsize=(4 * len(arm_geometry), 4))

# TODO Edit plot_arm_components to do matrix transforms
plot_arm_components(axs, arm_geometry, b_do_pose_matrix=False)

In [ ]:
grader.check("Step_1")

## Step 2 - set the matrices that rotate the links/palm

TODO: Edit **set_matrices_all_components** so that the "Matrix_pose" key has the rotation for each link

This is the equivalent of the **mat_rotate_link** matrices from the lecture activity.

Reminder that the plotting below can be used to check the results

In [ ]:
def set_rotation_matrices_all_components(arm_list, angles_list):
    """ For each component, set the Matrix_pose to be a matrix that rotates each component by the desired amount
    In other words, set the "Matrix_pose" key in each dictionary in arm_link

    In the Homework you'll edit a similar function to set "Matrix_pose" to be the matrix that takes the component to its final position
    @param arm_list - the list of the arm component dictionaries.
    @param angles_list - a list of angles, one for each link, one for the palm, and one for the fingers
    """

    # TODO: Set the "Matrix_pose" matrix for each arm component

    # The base link - it doesn't move, so this is the identity matrix. Setting here just to show the syntax of
    #   setting the Matrix_pose keye
    arm_list[0]["Matrix_pose"] = np.identity(3)

    # TODO:
    #  For each link, set Matrix_pose to be the rotation matrix that rotates the link to the correct orientation

    # This gets the angle that corresponds to each link plus the link dictionary for all links
    for ang, link in zip(angles_list[0:-1], arm_list[1:-1]):
        # Save the angle to the dictionary
        link["Angle"] = ang
        link["Matrix_pose"] = np.identity(3)
        # TODO 
        # For the Lab: Set the matrix in the key "Matrix_pose" to be a rotation for that link
        #  Yes, this should be a simple call to make_rotation_matrix
        ...

    # Now do the gripper (palm) - the fingers are optional
    # If you don't do the fingers, just use the same matrix for the palm AND the fingers
    gripper = arm_list[-1]
    angles_gripper = angles_list[-1]
    palm_obj = gripper[0]
    palm_obj["Angle"] = angles_gripper[0]
    palm_obj["Matrix_pose"] = np.identity(3)

    # TODO Set the matrix pose for the palm. This is exactly the same as setting the link pose. Note that the angle is stored
    # in palm["Angle"]
    ...

    # TODO (optional): Rotate the finger by the desired angle as well as the rotation by the palm angle
    #   If you do not do the optional finger rotation then the finger rotation matrix should be the same as the palm's
    #  Optional:
    #   Translate the base of the finger back to the origin, rotate it, then translate it back out
    #   Reminder: The middle of the finger can be found using mt.get_dx_dy_from_matrix
    #    Note 1: You want to move the base of the finger, NOT the middle, to the origin before you do the rotate
    #    Note 2: The top finger rotates by finger_angle, the bottom by - finger_angle - this is already handled
    #      in finger_angle
    for finger, finger_angle in zip(gripper[1:3], angles_gripper[1:]):
        finger["Angle"] = finger_angle
        ...


In [ ]:
# Call the above function with test values
angles_check = [np.pi/2, -np.pi/4, -3.0 * np.pi/4, [np.pi/3.0, np.pi/4.0, -np.pi/4.0]]

# You can use the above set of angles to check each part of the plot
#  Make sure to leave it on angles_check when you're done
set_rotation_matrices_all_components(arm_geometry, angles_check)


In [ ]:
# Check the rotation matrix for the first link
mat_rot_link1 = arm_geometry[1]["Matrix_pose"]
print(mat_rot_link1)
mat_rot_link1_check = np.array([[0.0, -1.0, 0.0], [1.0, 0.0, 0.0], [0.0, 0.0, 1.0]])
assert(np.all(np.isclose(mat_rot_link1, mat_rot_link1_check)))

In [ ]:
# Check the rotation matrix for the palm
mat_rot_palm = arm_geometry[-1][0]["Matrix_pose"]
print(mat_rot_palm)
mat_rot_palm_check = mt.make_rotation_matrix(angles_check[-1][0])
assert(np.all(np.isclose(mat_rot_palm, mat_rot_palm_check, atol=0.3)))

In [ ]:
# Check the rotation matrix for the top finger (optional)
mat_rot_top_finger = arm_geometry[-1][1]["Matrix_pose"]
print(mat_rot_top_finger)
mat_rot_top_finger_check = np.array([[-0.2588, -0.9659, 0.0050], [0.9659, -0.2588, 0.0379], [0.0, 0.0, 1.0]])
assert(np.all(np.isclose(mat_rot_top_finger, mat_rot_top_finger_check, atol=0.3)))

In [ ]:
# Check the rotation matrix for the bottom finger (optional)
mat_rot_bot_finger = arm_geometry[-1][2]["Matrix_pose"]
print(mat_rot_bot_finger)
mat_rot_bot_finger_check = np.array([[0.9659, -0.2588, 0.0304], [0.2588, 0.9659, 0.0233], [0.0, 0.0, 1.0]])
assert(np.all(np.isclose(mat_rot_bot_finger, mat_rot_bot_finger_check, atol=0.3)))

In [ ]:
grader.check("set_matrix_from_angles")

# Plotting for step 2

I've included several different sets of angle values for you.

In [ ]:
# Step 2 - rotate each link element in its own cooridinate system
# Several different angles to check your results with
angles_none = [0.0, 0.0, 0.0, [0.0, 0.0, 0.0]]
angles_check_fingers = [np.pi/2, -np.pi/4, -3.0 * np.pi/4, [0.0, np.pi/4.0, -np.pi/4.0]]
angles_check_wrist = [np.pi/2, -np.pi/4, -3.0 * np.pi/4, [np.pi/3.0, 0.0, 0.0]]
angles_check = [np.pi/2, -np.pi/4, -3.0 * np.pi/4, [np.pi/3.0, np.pi/4.0, -np.pi/4.0]]

# You can use the above set of angles to check each part of the plot
#  Make sure to leave it on angles_check when you're done
set_rotation_matrices_all_components(arm_geometry, angles_check)
fig, axs = plt.subplots(1, len(arm_geometry), figsize=(6 * len(arm_geometry), 6))
plot_arm_components(axs, arm_geometry, b_do_pose_matrix=True)

In [ ]:
# Manual grade
print("This is a manually-graded question; there is no grader.check() function. See rubric and slides for more information on expected output.")

## Hours and collaborators
Required for every assignment - fill out before you hand-in.

Listing names and websites helps you to document who you worked with and what internet help you received in the case of any plagiarism issues. You should list names of anyone (in class or not) who has substantially helped you with an assignment - or anyone you have *helped*. You do not need to list TAs.

Listing hours helps us track if the assignments are too long.

In [ ]:

# List of names (creates a set)
worked_with_names = {"not filled out"}
# List of URLS I25 (creates a set)
websites = {"not filled out"}
# Approximate number of hours, including lab/in-class time
hours = -1.5

In [ ]:
grader.check("hours_collaborators")

### To submit

Read me!!!   **Read me!!!**

- Submit this .ipynb file **AND** arm_routines.py. If you don't include arm_routines.py Gradescope cannot magically reach out to your computer and find your arm_routines.py file.
- We will supply matrix_routines.py for you (it won't break anything if you do include it)

If the Gradescope autograder fails, please check here first for common reasons for it to fail
    https://docs.google.com/presentation/d/1tYa5oycUiG4YhXUq5vHvPOpWJ4k_xUPp2rUNIL7Q9RI/edit?usp=sharing

Lots of people forget arm_routines.py. 